<a href="https://colab.research.google.com/github/rajy4683/EVAP2/blob/master/ENDS9_TF_commonqa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Mon Jan  4 07:46:31 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   62C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator

import spacy
import numpy as np
import pandas as pd 
import json
import random
import math
import time

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp /content/drive/MyDrive/EVA4/ENDS9/CommonQA.zip .

In [ ]:
!unzip /content/CommonQA.zip

Archive:  /content/CommonQA.zip
   creating: CommonQA/
  inflating: CommonQA/dev_rand_split.jsonl  
  inflating: CommonQA/test_rand_split_no_answers.jsonl  
  inflating: CommonQA/train_rand_split.jsonl  


Then set a random seed for deterministic results/reproducability.

In [ ]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [ ]:
read_file("/content/CommonQA/train_rand_split.jsonl", "dev")

In [ ]:
sentence = []
label = []
with open('/content/CommonQA/train_rand_split.jsonl') as h:
    for line in h:
        example = json.loads(line)
        scores = []
        merged_choices = ' A: '.join([choice['text'] for choice in example['question']['choices']])
        input = 'Q: ' + example['question']['stem'] + ' A: ' + merged_choices
        correct_answer = [ choice['text'] for choice in example['question']['choices'] if choice['label'] == example['answerKey'] ][0]
        sentence.append(input)
        label.append(correct_answer)
        #print(input, correct_answer)
dataset_df = pd.DataFrame({'sentence':sentence, 'label':label})

In [ ]:
dataset_df

,sentence,label
0,Q: The sanctions against the school were a pun...,ignore
1,Q: Sammy wanted to go to where the people were...,populated areas
2,Q: To locate a choker not located in a jewelry...,jewelry store
3,Q: Google Maps and other highway and street GP...,atlas
4,Q: The fox walked from the city into the fores...,natural habitat
...,...,...
9736,Q: What would someone need to do if he or she ...,telling all
9737,Q: Where might you find a chair at an office? ...,cubicle
9738,Q: Where would you buy jeans in a place with a...,shopping mall
9739,Q: John fell down the well. he couldn't belie...,fairytale


In [ ]:
#dataset = pd.read_csv("/content/dev.tsv", sep='\t', header=None)
dataset = pd.read_csv("/content/qasc/train.tsv", sep='\t', header=None)
dataset.columns = ["sentence", "label"]

In [ ]:
dataset

,sentence,label
0,What type of water formation is formed by clou...,beads
1,Where do beads of water come from? \n (A) Too ...,Vapor turning into a liquid
2,What forms beads of water? \n (A) Necklaces. ...,Steam.
3,what kind of beads are formed from vapor conde...,h2o
4,what kind of beads are formed by their vapor c...,h2o
...,...,...
8129,Chitin can be used for protection by whom? \n ...,Fish
8130,Which type of animal uses plates for protectio...,reptiles
8131,What are used for protection by fish? \n (A) s...,scales
8132,What are pangolins covered in? \n (A) tunicate...,protection


Instantiate our German and English spaCy models.

In [ ]:
!python3 -m spacy download de_core_news_sm

     |████████████████████████████████| 14.9MB 26.0MB/s 
  Created wheel for de-core-news-sm: filename=de_core_news_sm-2.2.5-cp36-none-any.whl size=14907057 sha256=159fc9a1398616216f21ed4d1d5c7f9bac1ae3aba32ea1f5a513c4f3886181ab
  Stored in directory: /tmp/pip-ephem-wheel-cache-_7ehpzcl/wheels/ba/3f/ed/d4aa8e45e7191b7f32db4bfad565e7da1edbf05c916ca7a1ca
Successfully built de-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')


In [ ]:
import de_core_news_sm

In [ ]:
spacy_de =  de_core_news_sm.load()
spacy_en = spacy.load('en')

Previously we reversed the source (German) sentence, however in the paper we are implementing they don't do this, so neither will we.

In [ ]:
def tokenize_de(text):
    """
    Tokenizes German text from a string into a list of strings
    """
    return [tok.text for tok in spacy_de.tokenizer(text)]

def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

Create our fields to process our data. This will append the "start of sentence" and "end of sentence" tokens as well as converting all words to lowercase.

In [ ]:
from torchtext import data 
import matplotlib.pyplot as plt
Sentence = data.Field(sequential = True, tokenize = 'spacy', batch_first =False, include_lengths=False, lower=False)
Label = data.Field(sequential =True, tokenize ='spacy', is_target=False, batch_first =False, include_lengths=False, lower=False)

In [ ]:
dataset = dataset_df
fields = [('sentence', Sentence),('label',Label)]
example = [data.Example.fromlist([dataset.sentence[i],dataset.label[i]], fields) for i in range(dataset.shape[0])] 
commonqa_ds = data.Dataset(example, fields)
(train, valid) = commonqa_ds.split(split_ratio=[0.85, 0.15], random_state=random.seed(SEED))

In [ ]:
Sentence.build_vocab(train)
Label.build_vocab(train)

In [ ]:
print('Size of input vocab : ', len(Sentence.vocab))
print('Size of label vocab : ', len(Label.vocab))
print('Top 10 words appreared repeatedly :', list(Sentence.vocab.freqs.most_common(10)))
print('Labels : ', Label.vocab.stoi)

Size of input vocab :  11736
Size of label vocab :  3515
Top 10 words appreared repeatedly : [(':', 49680), ('A', 41641), ('Q', 8280), ('?', 8228), ('to', 5004), ('a', 4761), ('the', 3593), ('what', 3290), (',', 3103), ('of', 2335)]
Labels :  defaultdict(<function _default_unk_index at 0x7f340adfbea0>, {'<unk>': 0, '<pad>': 1, 'store': 2, 'of': 3, 'house': 4, 'to': 5, 'office': 6, "'s": 7, 'room': 8, 'city': 9, 'get': 10, 'money': 11, 'building': 12, 'school': 13, 'go': 14, 'have': 15, 'new': 16, 'in': 17, 'music': 18, 'water': 19, 'home': 20, 'down': 21, 'food': 22, 'feel': 23, 'being': 24, 'park': 25, 'area': 26, 'restaurant': 27, 'shop': 28, 'good': 29, 'fun': 30, 'game': 31, 'out': 32, 'kitchen': 33, 'own': 34, 'cabinet': 35, 'countryside': 36, 'death': 37, 'car': 38, 'getting': 39, 'make': 40, 'people': 41, 'work': 42, 'play': 43, 'table': 44, 'up': 45, 'band': 46, 'desk': 47, 'feeling': 48, 'for': 49, 'market': 50, 'michigan': 51, 'station': 52, 'better': 53, 'ground': 54, 'hotel

In [ ]:
Sentence.vocab.stoi['new construction'], Label.vocab.stoi['new construction'] 

(0, 0)

Load our data.

Then create our vocabulary, converting all tokens appearing less than twice into `<unk>` tokens.

Finally, define the `device` and create our iterators.

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
BATCH_SIZE = 128

train_iterator_de_en, valid_iterator_de_en, test_iterator_de_en = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device)

In [ ]:
BATCH_SIZE=128
train_iterator, test_iterator = BucketIterator.splits(
    (train, valid), 
    batch_size = BATCH_SIZE, 
    device = device, sort=False, shuffle=False)

In [ ]:
iter_one = next(iter(train_iterator))

In [ ]:
iter_one.label.shape, 

(torch.Size([4, 128]),)

In [ ]:
for entry in (iter_one.sentence[0][0,:]):
    print(entry.item())

## Seq2Seq Model

Putting the encoder and decoder together, we get:

![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/seq2seq7.png?raw=1)

Again, in this implementation we need to ensure the hidden dimensions in both the encoder and the decoder are the same.

Briefly going over all of the steps:
- the `outputs` tensor is created to hold all predictions, $\hat{Y}$
- the source sequence, $X$, is fed into the encoder to receive a `context` vector
- the initial decoder hidden state is set to be the `context` vector, $s_0 = z = h_T$
- we use a batch of `<sos>` tokens as the first `input`, $y_1$
- we then decode within a loop:
  - inserting the input token $y_t$, previous hidden state, $s_{t-1}$, and the context vector, $z$, into the decoder
  - receiving a prediction, $\hat{y}_{t+1}$, and a new hidden state, $s_t$
  - we then decide if we are going to teacher force or not, setting the next input as appropriate (either the ground truth next token in the target sequence or the highest predicted next token)

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        
        self.embedding = nn.Embedding(input_dim, emb_dim) #no dropout as only one layer!
        
        self.rnn = nn.GRU(emb_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        text, text_lengths = src
        embedded = self.dropout(self.embedding(text))
        # packed_seq = nn.utils.rnn.pack_padded_sequence(embedded, 
        #                                                text_lengths.cpu(),
        #                                                batch_first=False,
        #                                                enforce_sorted=False)        
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, hidden = self.rnn(embedded) #no cell state! 
        ## outputs is a packed sequence but since it is not used we will not 
        ## unpack it         
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden

## Model Definition

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        
        self.embedding = nn.Embedding(input_dim, emb_dim) #no dropout as only one layer!
        
        self.rnn = nn.GRU(emb_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, hidden = self.rnn(embedded) #no cell state!
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden

class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        self.output_dim = output_dim
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.GRU(emb_dim + hid_dim, hid_dim)
        
        self.fc_out = nn.Linear(emb_dim + hid_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, context):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #context = [n layers * n directions, batch size, hid dim]
        
        #n layers and n directions in the decoder will both always be 1, therefore:
        #hidden = [1, batch size, hid dim]
        #context = [1, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        # print(embedded.shape, hidden.shape, context.shape)
        
        #embedded = [1, batch size, emb dim]
                
        emb_con = torch.cat((embedded, context), dim = 2)
            
        #emb_con = [1, batch size, emb dim + hid dim]
            
        output, hidden = self.rnn(emb_con, hidden)
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        #seq len, n layers and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        output = torch.cat((embedded.squeeze(0), hidden.squeeze(0), context.squeeze(0)), 
                           dim = 1)
        
        #output = [batch size, emb dim + hid dim * 2]
        
        prediction = self.fc_out(output)
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is the context
        context = self.encoder(src)
        
        #context also used as the initial hidden state of the decoder
        hidden = context
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden state and the context state
            #receive output tensor (predictions) and new hidden state
            output, hidden = self.decoder(input, hidden, context)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1

        return outputs

# Training the Seq2Seq Model

The rest of this session is very similar to the previous one. 

We initialise our encoder, decoder and seq2seq model (placing it on the GPU if we have one). As before, the embedding dimensions and the amount of dropout used can be different between the encoder and the decoder, but the hidden dimensions must remain the same.

In [ ]:
# INPUT_DIM = len(SRC.vocab)
# OUTPUT_DIM = len(TRG.vocab)

INPUT_DIM = len(Sentence.vocab)
OUTPUT_DIM = len(Label.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
ENC_DROPOUT = 0.2
DEC_DROPOUT = 0.2

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, DEC_DROPOUT)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = Seq2Seq(enc, dec, device).to(device)

Next, we initialize our parameters. The paper states the parameters are initialized from a normal distribution with a mean of 0 and a standard deviation of 0.01, i.e. $\mathcal{N}(0, 0.01)$. 

It also states we should initialize the recurrent parameters to a special initialization, however to keep things simple we'll also initialize them to $\mathcal{N}(0, 0.01)$.

In [ ]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean=0, std=0.01)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(11736, 256)
    (rnn): GRU(256, 512)
    (dropout): Dropout(p=0.2, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(3515, 256)
    (rnn): GRU(768, 512)
    (fc_out): Linear(in_features=1280, out_features=3515, bias=True)
    (dropout): Dropout(p=0.2, inplace=False)
  )
)

We print out the number of parameters.


In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 11,558,843 trainable parameters


We initiaize our optimizer.

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=0.0001)

We also initialize the loss function, making sure to ignore the loss on `<pad>` tokens.

In [ ]:
#TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

TRG_PAD_IDX = Label.vocab.stoi[Label.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

We then create the training loop...

In [ ]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.sentence
        trg = batch.label
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        # print("output_dim:", output.shape)
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        # print("output_dim before loss:", output.shape, trg.shape)
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        scheduler.step()
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

...and the evaluation loop, remembering to set the model to `eval` mode and turn off teaching forcing.

In [ ]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            # src = batch.src
            # trg = batch.trg
            src = batch.sentence
            trg = batch.label
            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

We'll also define the function that calculates how long an epoch takes.

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

Then, we train our model, saving the parameters that give us the best validation loss.

In [ ]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, test_iterator, criterion)
    
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 2s
	Train Loss: 7.747 | Train PPL: 2315.737
	 Val. Loss: 6.972 |  Val. PPL: 1066.356
Epoch: 02 | Time: 0m 2s
	Train Loss: 6.403 | Train PPL: 603.608
	 Val. Loss: 6.934 |  Val. PPL: 1026.484
Epoch: 03 | Time: 0m 2s
	Train Loss: 6.304 | Train PPL: 546.912
	 Val. Loss: 6.981 |  Val. PPL: 1076.070
Epoch: 04 | Time: 0m 2s
	Train Loss: 6.271 | Train PPL: 528.765
	 Val. Loss: 6.996 |  Val. PPL: 1091.783
Epoch: 05 | Time: 0m 2s
	Train Loss: 6.240 | Train PPL: 512.814
	 Val. Loss: 6.996 |  Val. PPL: 1092.124
Epoch: 06 | Time: 0m 2s
	Train Loss: 6.206 | Train PPL: 495.889
	 Val. Loss: 6.990 |  Val. PPL: 1085.880
Epoch: 07 | Time: 0m 2s
	Train Loss: 6.173 | Train PPL: 479.593
	 Val. Loss: 6.982 |  Val. PPL: 1077.588
Epoch: 08 | Time: 0m 2s
	Train Loss: 6.142 | Train PPL: 465.057
	 Val. Loss: 6.974 |  Val. PPL: 1068.542
Epoch: 09 | Time: 0m 2s
	Train Loss: 6.114 | Train PPL: 452.347
	 Val. Loss: 6.967 |  Val. PPL: 1060.803
Epoch: 10 | Time: 0m 2s
	Train Loss: 6.087 | Train PPL

In [ ]:
N_EPOCHS = 20
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, test_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 2s
	Train Loss: 6.061 | Train PPL: 428.615
	 Val. Loss: 6.968 |  Val. PPL: 1062.412
Epoch: 02 | Time: 0m 2s
	Train Loss: 6.031 | Train PPL: 416.110
	 Val. Loss: 6.974 |  Val. PPL: 1068.579
Epoch: 03 | Time: 0m 2s
	Train Loss: 6.001 | Train PPL: 403.660
	 Val. Loss: 6.971 |  Val. PPL: 1064.756
Epoch: 04 | Time: 0m 2s
	Train Loss: 5.964 | Train PPL: 389.241
	 Val. Loss: 6.948 |  Val. PPL: 1041.324
Epoch: 05 | Time: 0m 2s
	Train Loss: 5.924 | Train PPL: 373.858
	 Val. Loss: 6.908 |  Val. PPL: 999.754
Epoch: 06 | Time: 0m 2s
	Train Loss: 5.881 | Train PPL: 358.269
	 Val. Loss: 6.853 |  Val. PPL: 946.454
Epoch: 07 | Time: 0m 2s
	Train Loss: 5.832 | Train PPL: 340.925
	 Val. Loss: 6.802 |  Val. PPL: 899.902
Epoch: 08 | Time: 0m 2s
	Train Loss: 5.782 | Train PPL: 324.481
	 Val. Loss: 6.760 |  Val. PPL: 862.300
Epoch: 09 | Time: 0m 2s
	Train Loss: 5.728 | Train PPL: 307.236
	 Val. Loss: 6.720 |  Val. PPL: 828.767
Epoch: 10 | Time: 0m 2s
	Train Loss: 5.674 | Train PPL: 291.

In [ ]:
N_EPOCHS = 20
CLIP = 1

best_valid_loss = 6.280 #float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, test_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 2s
	Train Loss: 4.861 | Train PPL: 129.123
	 Val. Loss: 6.240 |  Val. PPL: 512.902
Epoch: 02 | Time: 0m 2s
	Train Loss: 4.778 | Train PPL: 118.886
	 Val. Loss: 6.199 |  Val. PPL: 492.367
Epoch: 03 | Time: 0m 2s
	Train Loss: 4.709 | Train PPL: 110.917
	 Val. Loss: 6.166 |  Val. PPL: 476.265
Epoch: 04 | Time: 0m 2s
	Train Loss: 4.635 | Train PPL: 103.028
	 Val. Loss: 6.127 |  Val. PPL: 458.265
Epoch: 05 | Time: 0m 2s
	Train Loss: 4.572 | Train PPL:  96.717
	 Val. Loss: 6.076 |  Val. PPL: 435.379
Epoch: 06 | Time: 0m 2s
	Train Loss: 4.504 | Train PPL:  90.380
	 Val. Loss: 6.041 |  Val. PPL: 420.411
Epoch: 07 | Time: 0m 2s
	Train Loss: 4.437 | Train PPL:  84.509
	 Val. Loss: 6.005 |  Val. PPL: 405.556
Epoch: 08 | Time: 0m 2s
	Train Loss: 4.375 | Train PPL:  79.443
	 Val. Loss: 5.974 |  Val. PPL: 392.916
Epoch: 09 | Time: 0m 2s
	Train Loss: 4.302 | Train PPL:  73.848
	 Val. Loss: 5.942 |  Val. PPL: 380.770
Epoch: 10 | Time: 0m 2s
	Train Loss: 4.238 | Train PPL:  69.284


In [ ]:
N_EPOCHS = 30
CLIP = 1

best_valid_loss = 5.709 #float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, test_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 2s
	Train Loss: 3.601 | Train PPL:  36.625
	 Val. Loss: 5.693 |  Val. PPL: 296.919
Epoch: 02 | Time: 0m 2s
	Train Loss: 3.560 | Train PPL:  35.177
	 Val. Loss: 5.681 |  Val. PPL: 293.331
Epoch: 03 | Time: 0m 2s
	Train Loss: 3.518 | Train PPL:  33.715
	 Val. Loss: 5.669 |  Val. PPL: 289.643
Epoch: 04 | Time: 0m 2s
	Train Loss: 3.460 | Train PPL:  31.827
	 Val. Loss: 5.651 |  Val. PPL: 284.434
Epoch: 05 | Time: 0m 2s
	Train Loss: 3.416 | Train PPL:  30.433
	 Val. Loss: 5.632 |  Val. PPL: 279.289
Epoch: 06 | Time: 0m 2s
	Train Loss: 3.371 | Train PPL:  29.111
	 Val. Loss: 5.625 |  Val. PPL: 277.139
Epoch: 07 | Time: 0m 2s
	Train Loss: 3.317 | Train PPL:  27.581
	 Val. Loss: 5.610 |  Val. PPL: 273.021
Epoch: 08 | Time: 0m 2s
	Train Loss: 3.268 | Train PPL:  26.251
	 Val. Loss: 5.599 |  Val. PPL: 270.048
Epoch: 09 | Time: 0m 2s
	Train Loss: 3.227 | Train PPL:  25.203
	 Val. Loss: 5.584 |  Val. PPL: 266.206
Epoch: 10 | Time: 0m 2s
	Train Loss: 3.182 | Train PPL:  24.087


In [ ]:
N_EPOCHS = 50
CLIP = 1

best_valid_loss = 5.484 #float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, test_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 2s
	Train Loss: 2.445 | Train PPL:  11.535
	 Val. Loss: 5.488 |  Val. PPL: 241.844
Epoch: 02 | Time: 0m 2s
	Train Loss: 2.425 | Train PPL:  11.305
	 Val. Loss: 5.480 |  Val. PPL: 239.949
Epoch: 03 | Time: 0m 2s
	Train Loss: 2.403 | Train PPL:  11.058
	 Val. Loss: 5.477 |  Val. PPL: 239.027
Epoch: 04 | Time: 0m 2s
	Train Loss: 2.374 | Train PPL:  10.738
	 Val. Loss: 5.471 |  Val. PPL: 237.608
Epoch: 05 | Time: 0m 2s
	Train Loss: 2.345 | Train PPL:  10.437
	 Val. Loss: 5.465 |  Val. PPL: 236.272
Epoch: 06 | Time: 0m 2s
	Train Loss: 2.324 | Train PPL:  10.217
	 Val. Loss: 5.460 |  Val. PPL: 235.171
Epoch: 07 | Time: 0m 2s
	Train Loss: 2.302 | Train PPL:   9.992
	 Val. Loss: 5.450 |  Val. PPL: 232.854
Epoch: 08 | Time: 0m 2s
	Train Loss: 2.273 | Train PPL:   9.711
	 Val. Loss: 5.454 |  Val. PPL: 233.587
Epoch: 09 | Time: 0m 2s
	Train Loss: 2.253 | Train PPL:   9.516
	 Val. Loss: 5.447 |  Val. PPL: 231.988
Epoch: 10 | Time: 0m 2s
	Train Loss: 2.221 | Train PPL:   9.218


In [ ]:
optimizer = optim.Adam(model.parameters(), lr=5e-6, weight_decay=0.0005)

In [ ]:
best_valid_loss

4.96515295902888

In [ ]:
N_EPOCHS = 50
CLIP = 1
model.load_state_dict(torch.load('tut2-model.pt'))
best_valid_loss = 4.9634 #float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, test_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')
print(best_valid_loss)

Epoch: 01 | Time: 0m 2s
	Train Loss: 2.041 | Train PPL:   7.702
	 Val. Loss: 4.956 |  Val. PPL: 141.985
Epoch: 02 | Time: 0m 2s
	Train Loss: 2.037 | Train PPL:   7.665
	 Val. Loss: 4.958 |  Val. PPL: 142.298
Epoch: 03 | Time: 0m 2s
	Train Loss: 2.030 | Train PPL:   7.611
	 Val. Loss: 4.958 |  Val. PPL: 142.280
Epoch: 04 | Time: 0m 2s
	Train Loss: 2.035 | Train PPL:   7.649
	 Val. Loss: 4.959 |  Val. PPL: 142.447
Epoch: 05 | Time: 0m 2s
	Train Loss: 2.033 | Train PPL:   7.634
	 Val. Loss: 4.960 |  Val. PPL: 142.554
Epoch: 06 | Time: 0m 2s
	Train Loss: 2.027 | Train PPL:   7.589
	 Val. Loss: 4.960 |  Val. PPL: 142.524
Epoch: 07 | Time: 0m 2s
	Train Loss: 2.032 | Train PPL:   7.628
	 Val. Loss: 4.960 |  Val. PPL: 142.545
Epoch: 08 | Time: 0m 2s
	Train Loss: 2.031 | Train PPL:   7.622
	 Val. Loss: 4.959 |  Val. PPL: 142.499
Epoch: 09 | Time: 0m 2s
	Train Loss: 2.027 | Train PPL:   7.590
	 Val. Loss: 4.960 |  Val. PPL: 142.523
Epoch: 10 | Time: 0m 2s
	Train Loss: 2.027 | Train PPL:   7.595


In [ ]:
optimizer = optim.Adam(model.parameters(), lr=5e-8, weight_decay=0.005)

In [ ]:
N_EPOCHS = 20
CLIP = 1
model.load_state_dict(torch.load('tut2-model.pt'))
best_valid_loss = 4.890922844409943 #float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, test_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')
print(best_valid_loss)

Epoch: 01 | Time: 0m 2s
	Train Loss: 2.241 | Train PPL:   9.402
	 Val. Loss: 4.890 |  Val. PPL: 132.906
Epoch: 02 | Time: 0m 2s
	Train Loss: 2.248 | Train PPL:   9.473
	 Val. Loss: 4.890 |  Val. PPL: 132.938
Epoch: 03 | Time: 0m 2s
	Train Loss: 2.255 | Train PPL:   9.537
	 Val. Loss: 4.890 |  Val. PPL: 132.961
Epoch: 04 | Time: 0m 2s
	Train Loss: 2.249 | Train PPL:   9.477
	 Val. Loss: 4.891 |  Val. PPL: 133.118
Epoch: 05 | Time: 0m 2s
	Train Loss: 2.259 | Train PPL:   9.572
	 Val. Loss: 4.891 |  Val. PPL: 133.125
Epoch: 06 | Time: 0m 2s
	Train Loss: 2.255 | Train PPL:   9.532
	 Val. Loss: 4.891 |  Val. PPL: 133.151
Epoch: 07 | Time: 0m 2s
	Train Loss: 2.262 | Train PPL:   9.598
	 Val. Loss: 4.892 |  Val. PPL: 133.192
Epoch: 08 | Time: 0m 2s
	Train Loss: 2.263 | Train PPL:   9.611
	 Val. Loss: 4.892 |  Val. PPL: 133.223
Epoch: 09 | Time: 0m 2s
	Train Loss: 2.267 | Train PPL:   9.647
	 Val. Loss: 4.892 |  Val. PPL: 133.251
Epoch: 10 | Time: 0m 2s
	Train Loss: 2.265 | Train PPL:   9.628


In [ ]:
N_EPOCHS = 20
CLIP = 1
model.load_state_dict(torch.load('/content/CommonQA.pt'))
best_valid_loss = 4.890922844409943 #float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, test_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')
print(best_valid_loss)

Epoch: 01 | Time: 0m 2s
	Train Loss: 2.241 | Train PPL:   9.402
	 Val. Loss: 4.890 |  Val. PPL: 132.904
Epoch: 02 | Time: 0m 2s
	Train Loss: 2.242 | Train PPL:   9.415
	 Val. Loss: 4.890 |  Val. PPL: 132.907
Epoch: 03 | Time: 0m 2s
	Train Loss: 2.249 | Train PPL:   9.474
	 Val. Loss: 4.890 |  Val. PPL: 132.911
Epoch: 04 | Time: 0m 2s
	Train Loss: 2.249 | Train PPL:   9.475
	 Val. Loss: 4.890 |  Val. PPL: 132.913
Epoch: 05 | Time: 0m 2s
	Train Loss: 2.248 | Train PPL:   9.470
	 Val. Loss: 4.890 |  Val. PPL: 132.915
Epoch: 06 | Time: 0m 2s
	Train Loss: 2.247 | Train PPL:   9.463
	 Val. Loss: 4.890 |  Val. PPL: 132.918
Epoch: 07 | Time: 0m 2s
	Train Loss: 2.245 | Train PPL:   9.439
	 Val. Loss: 4.890 |  Val. PPL: 132.920
Epoch: 08 | Time: 0m 2s
	Train Loss: 2.248 | Train PPL:   9.465
	 Val. Loss: 4.890 |  Val. PPL: 132.924
Epoch: 09 | Time: 0m 2s
	Train Loss: 2.242 | Train PPL:   9.415
	 Val. Loss: 4.890 |  Val. PPL: 132.927
Epoch: 10 | Time: 0m 2s
	Train Loss: 2.252 | Train PPL:   9.511


Finally, we test the model on the test set using these "best" parameters.

In [ ]:
model.load_state_dict(torch.load('tut2-model.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 10.525 | Test PPL: 37253.098 |


Just looking at the test loss, we get better performance. This is a pretty good sign that this model architecture is doing something right! Relieving the information compression seems like the way forard, and in the next tutorial we'll expand on this even further with *attention*.